# 3rd Homework of ADM



## Group members

### 1. Gabriele Tromboni, 2088799, tromboni.2088799@studenti.uniroma1.it
### 2. Claudiu Gabriel Gheorghiu, 1845227, gheorghiu.1845227@studenti.uniroma1.it 
### 3. Arianna Celli, 2067150, celli.2067150@studenti.uniroma1.it

### Importing useful packages


In [1]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import os
from pathlib import Path
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
import pandas as pd
from math import log
from heapq import heappop, heappush  
from tqdm import tqdm

# 1. Data collection

### As a first thing, we need to search specific informations towards a topic related to the most popular places to visit which are in this site: 'https://www.atlasobscura.com/places?sort=likes_count'.

In [2]:
paginaprincipale = requests.get('https://www.atlasobscura.com/places?sort=likes_count')

##### Here we send a request to get the content of a specific webpage. We wanted the content of the pages of the Most Popular Places in AtlasObscura website.

In [3]:
paginaprincipale

<Response [200]>

##### As we have the code '200' as the status of the reponse, so it means that we have successfully received the content of the desired webpage.

##### show the content of the webpage

In [5]:
paginaprincipalesoup = BeautifulSoup(paginaprincipale.content,'html.parser')

In [6]:
print(paginaprincipalesoup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="reload" name="turbo-visit-control"/>
  <script src="https://cdn.cookielaw.org/consent/c7042f21-9a02-4cf6-ad42-3e1f237e2b50/OtAutoBlock.js" type="text/javascript">
  </script>
  <script charset="UTF-8" data-domain-script="c7042f21-9a02-4cf6-ad42-3e1f237e2b50" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() { }
  </script>
  <link as="font" crossorigin="" href="https://fonts.atlasobscura.com/2/Platform-Regular-Web.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://fonts.atlasobscura.com/2/Platform-Medium-Web.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://fonts.atlasobscura.com/2/FreigTexProBookWeb.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://fonts.atlasobscura.com/2/FreigTexProBookItWeb.woff2" rel="pre

##### Here we will take a look at how many places we have in each page of the webpage. 

In [9]:
numplaces = list(paginaprincipalesoup.find_all('div', {'class': 'col-md-4 col-sm-6 col-xs-12'}))
len(numplaces)

18

##### As we know the number of places in each page is 18. 

##### Now we should get the url corresponding to each place. 

##### We found out that this information is in 'a' tag, which its class is "content-card content-card-place".

##### This is a first view of  the first place in the webpage. 

In [10]:
a = paginaprincipalesoup.find('a', {'class': 'content-card content-card-place'})
print(a)
url = a['href']
url

<a class="content-card content-card-place" data-physical-status="" data-place-id="1212" href="/places/city-hall-station">
<figure class="content-card-figure js-content-card-figure">
<img alt="" class="lazy img-responsive content-card-img" data-height="671" data-src="https://img.atlasobscura.com/8W4CtDGanZHSDesY4UBadY5_3p8yHfzzgicnzCMIJlY/rs:fill:372:248:1/g:ce/q:81/sm:1/scp:1/ar:1/aHR0cHM6Ly9hdGxh/cy1kZXYuczMuYW1h/em9uYXdzLmNvbS91/cGxvYWRzL3BsYWNl/X2ltYWdlcy8zYmNi/ZDg3MDIzMWIzZjcz/NTVfNTE1OTE5MTI5/MF85ODA0ZWE0MmIy/X28uanBn.jpg" data-width="1011" nopin="nopin" src="https://assets.atlasobscura.com/assets/blank-11b9c95a68e295dddd0ea924647536578ce285b2c8469a223c01df1ff3166af1.png"/>
</figure>
<div class="content-card-text">
<div class="detail-sm place-card-location">New York, New York</div>
<h3 class="title-md content-card-title"><span class="title-underline js-title-content">City Hall Station</span></h3>
<div class="subtitle-sm content-card-subtitle js-subtitle-content">A beautiful and ab

'/places/city-hall-station'

# 1.1. Get the list of places

##### Now here we want to get the urls of all the places in the first 400 pages. We write this urls in a txt file named places.

In [2]:
File_URLs = open("places.txt", "w")

for i in tqdm(range(1,401)):
    pages = 'https://www.atlasobscura.com/places?page=' + str(18*i) + '&sort=likes_count'
    response = requests.get(pages)
    Most_Popular_Places = BeautifulSoup(response.content, 'lxml')
    for item in Most_Popular_Places.find_all('a', class_='content-card content-card-place'):
        File_URLs.write(item.get('href')+ '\n')
        
File_URLs.close()

100%|██████████| 400/400 [34:46<00:00,  5.22s/it]  


# 1.2. Crawl places

#### Create folder in which we'll put all the page's subfolders


In [ ]:
os.mkdir(f'MostPopularPlaces')

#### Recall  file.txt in which there are the URLs of the places.

In [2]:
links_of_places = pd.read_csv('places.txt', header=None)

#### Creating subfolders to store the html files of the 400 pages in Most Popular Places website.

#### Separate the download to store html pages(This is the code for the purpose).


In [ ]:
perm_session = requests.Session()

for page in tqdm(range(400)):
    os.mkdir(f'MostPopularPlaces/MPP_page_{page + 1}')
    for place in range(0,18):
        with open(f'MostPopularPlaces/MPP_page_{page + 1}/place_{(18 * page) + place +1}.html', 'x', encoding = "utf-8") as f:
            url = 'https://www.atlasobscura.com' + links_of_places.iloc[(18 * page) + place][0]
            response = perm_session.get(url)
            html_page = response.text
            f.write(html_page)

###### We had to divide all downloads cause of many crash and mistakes in the requests of the webpage!!

# 1.3 Parse downloaded pages

#### At this point, we have all the HTML documents about the places of interest, and we can start to extract the places' information. The list of the information we desire for each place and their format is as follows:

In [2]:
Name = []
Tags = []
PeopleVisited = []
PeopleWant = []
Desc = []
ShortDesc = []
Nearby = []
Address = []
Alt = []
Long = []
Editors = []
PubDate = []
RelatedLists = []
RelatedPlaces = []
URL = []

#### We go through each HTML file that we have downloaded. 

#### Then we extract the information that we need from the HTML file and store them in a .tsv file towards a pandas DataFrame 

In [3]:
for i in tqdm(range(1,401)):
    for j in range(18*(i-1)+1, (18*i + 1)):
        f = open("/Users/gtrom/OneDrive/Desktop/ADM-HMK3/MostPopularPlaces/MPP_page_" + str(i) + '/place_' + str(j) + '.html', 'r', encoding = "utf8")
        page = BeautifulSoup(f, 'lxml')

    # Extracting the information:

    #1. placeName 
        try:
            placeName= page.find('h1', {'class':'DDPage__header-title'}).contents[0]
            Name.append(placeName)
        except:
            Name.append('')

    #2. placeTags
        try:
            tag= page.find_all('a', {'class': 'itemTags__link js-item-tags-link'})
            tag = [m.get('href').split('/categories/')[1] for m in tag]
            Tags.append(tag)
        except:
            Tags.append('')

    #3. numPeopleVisited
        try:
            numPeopleVisited=int(page.find_all('div', {'class':'title-md item-action-count'})[0].contents[0])
            PeopleVisited.append(numPeopleVisited)
        except:
            PeopleVisited.append('')

    #4. numPeopleWant
        try:    
            numPeopleWant=int(page.find_all('div', {'class':'title-md item-action-count'})[1].contents[0])
            PeopleWant.append(numPeopleWant)
        except:
            PeopleWant.append('')

    #5. Description
        try:
            Description=page.find_all('div',{'class':'DDP__body-copy'})[0].getText().strip().replace('\n', ' ').replace('\xa0', ' ')
            Desc.append(Description)
        except:
            Desc.append('')

    #6. GetShortDesc
        try:
            GetShortDesc= page.find_all('h3', {'class':'DDPage__header-dek'})[0].contents[0].replace('\n', ' ').replace('\xa0', ' ')
            ShortDesc.append(GetShortDesc)         
        except:
            ShortDesc.append('') 

    #7. placeNearby
    
        try:
            pnb = page.find_all('div', {'class': 'DDPageSiderailRecirc__item-title'})
            pnb = [i.get_text() for i in pnb]
            Nearby.append(pnb)
        except: 
            Nearby.append('')

    #8. placeAddress
        try:
            addr = page.find_all('address', {'class': 'DDPageSiderail__address'})[0]
            addr = addr.find('div').get_text(separator=' ').split('\n')[0]
            Address.append(addr)
        except:
            Address.append('')

    #9. placeAlt
        try:
            alt_long = page.find('div', {'class': 'DDPageSiderail__coordinates js-copy-coordinates'}).contents[2].replace('\n','')
            al = alt_long.split(',')
            placeAlt = float(al[0])
            Alt.append(placeAlt)           
        except:
            Alt.append('') 

    #10. placeLong
        try:
            alt_long = page.find('div', {'class': 'DDPageSiderail__coordinates js-copy-coordinates'}).contents[2].replace('\n','')
            al = alt_long.split(',')
            placeLong = float(al[1])
            Long.append(placeLong)
        except:
            Long.append('')

        #11. placeEditors
        try:
            pe = page.find_all('div', {'class': 'ugc-editor-icons'})
            pe=[i.findChildren('a')[0].get('href') for i in pe]
            pe=[j.split('/users/')[1] for j in pe]
            Editors.append(pe)
        except: 
            Editors.append('')

        #12. placePubDate
        try:
            date = page.find('div', {'class': "DDPContributor__name"}).getText()
            date = pd.to_datetime(date)
            PubDate.append(date)
        except:
            PubDate.append('')

        #13. placeRelatedLists
        
        try:
            prl = page.find_all('div', {'data-gtm-template': 'DDP Footer Recirc Lists'})[0]
            prl = [i.text.strip() for i in prl.find_all('h3')]
            RelatedLists.append(prl)
        except:
            RelatedLists.append('')
    
        #14. placeRelatedPlaces
        
        try:
            prp = page.find_all('div', {'class':'CardRecircSection__title'})
            prp = [i for i in prp if i.text == 'Related Places'][0]
            prp = prp.find_all_next('div',{'class': 'Card__action-btns vue-js-been-there-everywhere-place'})
            prp = [j.get('data-place-title') for j in prp]
            RelatedPlaces.append(prp)
        except:
            RelatedPlaces.append('')

        #15. placeURL
        try:
            url = page.find('meta', attrs = {'property':'og:url'})
            url = url['content']
            URL.append(url)
        except:
             URL.append('')

100%|██████████| 400/400 [14:01<00:00,  2.10s/it]  


In [5]:
df_places = pd.DataFrame(columns = [['placeName','placeTags', 'numPeopleVisited', 'numPeopleWant',
                                           'placeDesc', 'placeShortDesc', 'placeNearby', 'placeAddress',
                                           'placeAlt', 'placeLong', 'placeEditors', 'placePubDate', 'placeRelatedLists', 
                                           'placeRelatedPlaces', 'placeURL']])
df_places.placeName = Name
df_places.placeTags= Tags
df_places.numPeopleVisited = PeopleVisited
df_places.numPeopleWant= PeopleWant
df_places.placeDesc = Desc
df_places.placeShortDesc = ShortDesc
df_places.placeNearby = Nearby
df_places.placeAddress = Address
df_places.placeAlt = Alt
df_places.placeLong = Long
df_places.placeEditors = Editors
df_places.placePubDate = PubDate
df_places.placeRelatedLists = RelatedLists
df_places.placeRelatedPlaces = RelatedPlaces
df_places.placeURL = URL


C:\Users\gtrom\AppData\Local\Temp\ipykernel_20752\895549063.py:16: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_places.placePubDate = PubDate


#### Strore data into a file .tsv

In [6]:
data = df_places.to_csv('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/Mostpopularplaces2.tsv', sep = '\t', index = False)

#### Reload the file as a DataFrame

In [2]:
df_places = pd.read_csv('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/Mostpopularplaces2.tsv', sep = '\t')
df_places.head(5)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1828,8583,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['rebekah-otto', 'jrc2692']",2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2618,8182,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,['ctlssnghts'],2014-08-09,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,"['markets', 'film-locations', 'film']",3135,7570,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['meghanneal', 'julius-spada']",2016-08-01,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...",2429,7408,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'mark-casey']",2008-11-21,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...",4447,7046,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['cpilgrim', 'jarednagel']",2009-02-13,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",http://www.atlasobscura.com/places/catacombes-...


# 2. Search Engine

#### Now, we want to create two different Search Engines that, given as input a query, return the places that match the query.

#### First, you must pre-process all the information collected for each place by:
1. Removing stopwords
1. Removing punctuation
1. Stemming
1. Anything else it's needed

In [3]:
#nltk.download('stopwords')
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from nltk import punkt
#import string

In [7]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gtrom\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
df_places.head(5)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1828,8583,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['rebekah-otto', 'jrc2692']",2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2618,8182,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,['ctlssnghts'],2014-08-09,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,"['markets', 'film-locations', 'film']",3135,7570,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['meghanneal', 'julius-spada']",2016-08-01,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...",2429,7408,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'mark-casey']",2008-11-21,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...",4447,7046,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['cpilgrim', 'jarednagel']",2009-02-13,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",http://www.atlasobscura.com/places/catacombes-...


#### This is the dataframe will be cleaned on 'placeCleanDesc'

In [5]:
df_cleaned = df_places[['placeName', 'placeDesc', 'placeURL']].rename(columns = {'placeDesc' : 'placeCleanDesc'})
df_cleaned

,placeName,placeCleanDesc,placeURL
0,City Hall Station,The first New York City subway was built and o...,http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"Opened in 1839, Highgate is one of London’s mo...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,The ornate 19th-century painted roof and cobbl...,http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"Located on a jetty in the San Francisco Bay, t...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"In 2004, Parisian police were assigned to do a...",http://www.atlasobscura.com/places/catacombes-...
...,...,...,...
7195,Athenry Dominican Priory,"Located east of Galway city, the small town of...",http://www.atlasobscura.com/places/athenry-dom...
7196,The Cairo,Washington D.C.’s residential neighborhoods ar...,http://www.atlasobscura.com/places/the-cairo
7197,Winganon Space Capsule,Any out of town-ers driving along the dull str...,http://www.atlasobscura.com/places/winganon-sp...
7198,Cook & Book,Cook & Book in Brussels is far more than just ...,http://www.atlasobscura.com/places/cook-book


### Cleaning

In [6]:
stopWords = stopwords.words('english')
porter = PorterStemmer()

In [7]:
def clean_data(dataframe):
    dataframe.loc[ : , 'placeCleanDesc'] = dataframe.loc[ : , 'placeCleanDesc'].str.lower()
    dataframe.loc[ : , 'placeCleanDesc'] = dataframe.loc[ : , 'placeCleanDesc'].apply(word_tokenize)
    dataframe.loc[ : , 'placeCleanDesc'] = dataframe.loc[ :, 'placeCleanDesc'].apply(lambda row:[w for w in row if w.isalnum()])
    dataframe.loc[ : , 'placeCleanDesc'] = dataframe.loc[ :, 'placeCleanDesc'].apply(lambda row: [w for w in row if w not in stopWords])
    dataframe.loc[ : , 'placeCleanDesc'] = dataframe.loc[ :, 'placeCleanDesc'].apply(lambda row: [porter.stem(w) for w in row])

    return dataframe

In [8]:
df_cleaned = clean_data(df_cleaned)
df_cleaned

,placeName,placeCleanDesc,placeURL
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",http://www.atlasobscura.com/places/catacombes-...
...,...,...,...
7195,Athenry Dominican Priory,"[locat, east, galway, citi, small, town, athen...",http://www.atlasobscura.com/places/athenry-dom...
7196,The Cairo,"[washington, residenti, neighborhood, typifi, ...",http://www.atlasobscura.com/places/the-cairo
7197,Winganon Space Capsule,"[drive, along, dull, stretch, road, talala, wi...",http://www.atlasobscura.com/places/winganon-sp...
7198,Cook & Book,"[cook, book, brussel, far, bookstor, size, sup...",http://www.atlasobscura.com/places/cook-book


## 2.1. Conjunctive query
For the first version of the search engine, we narrow our interest to the description of each place. It means that you will evaluate queries only concerning the place's description.

Note: You should use the longer description placeDesc column and not the short description placeShortDesc.

### 2.1.1) Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

{
term_id_1:[document_1, document_2, document_4],

term_id_2:[document_1, document_3, document_5, document_6],
...}
where document_i is the id of a document that contains that specific word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

### Create the vocabulary

In [12]:
vocabulary = set()
for word in df_cleaned['placeCleanDesc']:
    vocabulary.update(word)

In [14]:
voc = {}
value = 0
for key in vocabulary:
    voc[key] = value 
    value += 1

##### Take a look

In [15]:
print(voc)

{'send': 0, 'westinghous': 1, 'roessel': 2, 'cultur': 3, 'mouro': 4, '50m': 5, 'leve': 6, 'mcgillivray': 7, 'physic': 8, 'krew': 9, 'napoleon': 10, 'unstuck': 11, 'tini': 12, 'cameraman': 13, 'sandman': 14, 'derid': 15, 'keyboardist': 16, 'schlachthof': 17, 'subsum': 18, 'umbrella': 19, 'romantiqu': 20, 'jam': 21, 'leac': 22, 'mealt': 23, 'lila': 24, 'skye': 25, 'lafayett': 26, 'niš': 27, 'fanat': 28, 'fern': 29, 'bellevil': 30, 'turnov': 31, 'winsom': 32, 'convent': 33, 'steepest': 34, 'receiv': 35, 'fondo': 36, '1967': 37, 'augustinian': 38, 'butteri': 39, 'hourli': 40, 'culde': 41, 'skyrocket': 42, 'tokangawhā': 43, 'unsecur': 44, 'blaskó': 45, 'buffi': 46, 'xitl': 47, 'spade': 48, 'spaniard': 49, 'elicit': 50, 'quarterdeck': 51, 'barreda': 52, 'midair': 53, 'splendor': 54, 'garcet': 55, 'misguidedli': 56, 'koni': 57, 'simbel': 58, 'húnaflói': 59, 'electrostat': 60, 'unguard': 61, 'guitar': 62, 'vocabulari': 63, 'iconostasi': 64, 'brick': 65, 'kaijihama': 66, 'b': 67, 'bldg': 68, 'g

###### Now import 'pickle' package for save our vocabulary and future files like the inverted index!

In [9]:
import pickle

In [ ]:
with open('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/vocabulary.pkl', 'wb') as f:
    pickle.dump(voc, f)

In [10]:
with open('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/vocabulary.pkl', 'rb') as f:
    voc = pickle.load(f)

###### An arrangement to build our first inverted index

In [9]:
df_cleaned['index'] = [i for i in range(0,7200)]

In [10]:
df_cleaned

,placeName,placeCleanDesc,placeURL,index
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",http://www.atlasobscura.com/places/city-hall-s...,0
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",http://www.atlasobscura.com/places/highgate-ce...,1
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",http://www.atlasobscura.com/places/leadenhall-...,2
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",http://www.atlasobscura.com/places/wave-organ,3
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",http://www.atlasobscura.com/places/catacombes-...,4
...,...,...,...,...
7195,Athenry Dominican Priory,"[locat, east, galway, citi, small, town, athen...",http://www.atlasobscura.com/places/athenry-dom...,7195
7196,The Cairo,"[washington, residenti, neighborhood, typifi, ...",http://www.atlasobscura.com/places/the-cairo,7196
7197,Winganon Space Capsule,"[drive, along, dull, stretch, road, talala, wi...",http://www.atlasobscura.com/places/winganon-sp...,7197
7198,Cook & Book,"[cook, book, brussel, far, bookstor, size, sup...",http://www.atlasobscura.com/places/cook-book,7198


### First Inverted Index

In [21]:
inverted_index = {}

for key,value in tqdm(voc.items()):
    inverted_index[value] = list(df_cleaned[df_cleaned.placeCleanDesc.apply(lambda row: key in row)]['index'])

100%|██████████| 39143/39143 [12:54<00:00, 50.57it/s]


In [23]:
with open('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/inverted_index.pkl', 'wb') as f:
    pickle.dump(inverted_index, f)

In [11]:
with open('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

In [12]:
inverted_index

{0: [15,
  122,
  178,
  258,
  452,
  475,
  500,
  624,
  625,
  701,
  880,
  894,
  946,
  1033,
  1088,
  1090,
  1110,
  1151,
  1229,
  1231,
  1242,
  1418,
  1515,
  1761,
  1772,
  1781,
  1809,
  1868,
  1871,
  2000,
  2015,
  2020,
  2083,
  2084,
  2150,
  2226,
  2267,
  2294,
  2358,
  2464,
  2566,
  2570,
  2615,
  2656,
  2680,
  2965,
  3072,
  3075,
  3081,
  3223,
  3226,
  3241,
  3317,
  3332,
  3333,
  3371,
  3427,
  3878,
  3902,
  3908,
  3956,
  3993,
  4064,
  4099,
  4138,
  4204,
  4378,
  4407,
  4428,
  4551,
  4567,
  4756,
  5042,
  5064,
  5364,
  5394,
  5401,
  5419,
  5491,
  5588,
  5610,
  5636,
  5684,
  5703,
  5749,
  5770,
  5773,
  5978,
  6149,
  6151,
  6323,
  6349,
  6395,
  6450,
  6468,
  6511,
  6587,
  6721,
  6881,
  6904,
  7109,
  7196],
 1: [1072, 4857, 4868],
 2: [2738],
 3: [28,
  29,
  32,
  45,
  62,
  110,
  117,
  124,
  130,
  142,
  143,
  156,
  160,
  173,
  176,
  181,
  182,
  188,
  206,
  224,
  252,
  287,
  300,

### 2.1.2) Execute the query
Given a query input by the user, for example:

american museum

The Search Engine is supposed to return a list of documents.

What documents do we want?
Since we are dealing with conjunctive queries (AND), each returned document should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

placeName

placeDesc

placeURL

### Search engine for the query in the example. 

##### If you want you can change it.

In [13]:
query = input(str)

<class 'str'>american museum


#### Clean query thanks to nltk library

In [14]:
def clean_query(query):
    
    cleaned_query = query.lower()
    cleaned_query = word_tokenize(cleaned_query)
    cleaned_query = [w for w in cleaned_query if w.isalnum()]
    cleaned_query = [w for w in cleaned_query if w not in stopWords]
    cleaned_query = [porter.stem(w) for w in cleaned_query]
    
    return cleaned_query

In [15]:
cleaned_query = clean_query(query)
cleaned_query

['american', 'museum']

In [16]:
def match_query(query):
    
    query_index = []
    
    for word in clean_query(query):
        if word in voc.keys():
            query_index.append(voc[word])
        else:
            print(f"ERROR: This query is not in the vocabulary, so you have to correct the request")
    
    return query_index
    

In [17]:
map_query =match_query(query)
map_query

[22851, 21370]

### First Search Engine

In [18]:
def search_engine(query):
    
    documents = [set(inverted_index[key]) for key in match_query(query) if key in inverted_index.keys()]
    results = set.intersection(*documents)

    return results

In [19]:
results = search_engine(query)
results

{59,
 141,
 173,
 224,
 320,
 329,
 356,
 360,
 415,
 690,
 783,
 788,
 887,
 913,
 954,
 970,
 985,
 1003,
 1039,
 1050,
 1072,
 1086,
 1097,
 1101,
 1126,
 1134,
 1158,
 1206,
 1211,
 1233,
 1246,
 1262,
 1329,
 1360,
 1363,
 1386,
 1400,
 1407,
 1423,
 1482,
 1541,
 1551,
 1552,
 1554,
 1580,
 1589,
 1623,
 1700,
 1741,
 1762,
 1781,
 1796,
 1797,
 1800,
 1861,
 1867,
 1890,
 1891,
 1962,
 1990,
 1991,
 2052,
 2077,
 2081,
 2088,
 2105,
 2133,
 2172,
 2193,
 2217,
 2218,
 2233,
 2269,
 2285,
 2287,
 2355,
 2419,
 2473,
 2556,
 2571,
 2580,
 2607,
 2656,
 2658,
 2666,
 2688,
 2734,
 2737,
 2746,
 2793,
 2834,
 2937,
 3002,
 3045,
 3082,
 3088,
 3168,
 3177,
 3198,
 3254,
 3255,
 3358,
 3364,
 3375,
 3400,
 3409,
 3418,
 3428,
 3432,
 3535,
 3611,
 3691,
 3692,
 3711,
 3712,
 3759,
 3769,
 3807,
 3808,
 3809,
 3828,
 3883,
 3958,
 4002,
 4033,
 4070,
 4076,
 4087,
 4093,
 4263,
 4289,
 4303,
 4311,
 4380,
 4395,
 4403,
 4406,
 4428,
 4442,
 4452,
 4457,
 4459,
 4465,
 4479,
 4483,
 45

In [20]:
def match_documents(query):
    
    indexes = search_engine(query)
    col = ['placeName', 'placeDesc', 'placeURL']
    output = pd.DataFrame(columns= col, index = indexes)
    
    output.placeName = [df_places.placeName.loc[i] for i in indexes]
    output.placeDesc = [df_places.placeDesc.loc[i] for i in indexes]
    output.placeURL = [df_places.placeURL.loc[i] for i in indexes]
    
    return output

### Here are the documents that matches with the query given.

#### All places where an 'american museum' is.

In [21]:
match_documents(query)

,placeName,placeDesc,placeURL
2052,Soumaya Museum,"Housing a whopping 66,000 pieces of predominan...",http://www.atlasobscura.com/places/soumaya-museum
1541,National Museum of Health and Medicine,"Once housed in downtown Washington, D.C., the ...",http://www.atlasobscura.com/places/national-mu...
6154,"Martha, the Last Passenger Pigeon",When Europeans began settling in the New World...,http://www.atlasobscura.com/places/martha-the-...
2571,Lunch Box Museum,Lunch boxes bring back a certain sense of nost...,http://www.atlasobscura.com/places/lunch-box-m...
3082,Museum of Russian Culture,"When you think San Francisco, you probably don...",http://www.atlasobscura.com/places/museum-of-r...
...,...,...,...
7156,Akin Free Library,"The Akin Free Library, a gracious stone struct...",http://www.atlasobscura.com/places/akin-free-l...
7157,Friendship Park,"On most days, San Diego’s Friendship Park is a...",http://www.atlasobscura.com/places/friendship-...
4087,McNutt Sculpture Garden,A hidden sculpture garden exists just off the ...,http://www.atlasobscura.com/places/mcnutt-scul...
2556,The Wolfsonian-FIU,The Wolfsonian-Florida International Universit...,http://www.atlasobscura.com/places/wolfsonian-...


## 2.2) Conjunctive query & Ranking score
For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contain all the words in the query.
Sort them by their similarity with the query.
Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.
To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the placeDesc. Let's see how.

In [22]:
df_cleaned.head(10)

,placeName,placeCleanDesc,placeURL
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",http://www.atlasobscura.com/places/catacombes-...
5,The Evolution Store,"[evolut, stand, among, cloth, store, restaur, ...",http://www.atlasobscura.com/places/evolution-n...
6,Magowan's Infinite Mirror Maze,"[peopl, stumbl, labyrinth, arm, outstretch, ev...",http://www.atlasobscura.com/places/magowans-in...
7,Underground Tunnels of Los Angeles,"[rest, countri, forc, go, dri, underneath, dow...",http://www.atlasobscura.com/places/underground...
8,Secret Tiled Staircase,"[16th, avenu, staircas, 163, step, mosaic, til...",http://www.atlasobscura.com/places/secret-tile...
9,Dream House,"[walk, church, street, tribeca, keep, eye, bla...",http://www.atlasobscura.com/places/dream-house


### 2.2.1) Inverted index
Your second Inverted Index must be of this format:

{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],

term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}

Practically, for each word, you want the list of documents in which it is contained and the relative tfIdf score.

Tip: TfIdf values are invariant for the query. Due to this reason, you can precalculate and store them accordingly.

### For this second search engine we have to calculate a second inverted index as we must also weigh the importance of the terms in the various documents through the so-called $tfidf$ $score$

### Second Inverted Index

In [40]:
from numpy.linalg import norm
import numpy as np

tf={}  #number of occurences of a word in all documents (term frequency)

for i in range(len(df_cleaned.placeCleanDesc)):
    tf[i] = {}
    for w in (df_cleaned.placeCleanDesc)[i]:
        if w not in tf[i]:
            tf[i][w]=0
        tf[i][w]+=1

# we have to weigh the terms in all documents by their inverse-document frequency based on the amount of documents we have and
# how many documents contains that term

idf= {term_id: len(docs) for term_id, docs in inverted_index.items()} # Inverse-document frequency

for key in idf:
    idf[key] =  np.log10(len(df_cleaned.placeCleanDesc)/idf[key])

# the second inverted index, we have changed the format as a dictionary of dictionaries to semplify our future research
# keys are the term_id, the values are a dict in which the keys are the documents and the values are the tfidf scores 

inverted_index_2 = {}
for j in tqdm(range(len(df_cleaned.placeCleanDesc))):
    for w in tf[j]:
        term_id = voc[w]
        if term_id not in list(inverted_index_2.keys()):
            inverted_index_2[term_id] = {}
        inverted_index_2[term_id][j] = tf[j][w] / idf[term_id]

100%|██████████| 7200/7200 [04:04<00:00, 29.45it/s]


In [43]:
with open('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/inverted_index_2.pkl', 'wb') as f:
    pickle.dump(inverted_index_2, f)

In [23]:
with open('C://Users//gtrom//OneDrive//Desktop//ADM-HMK3/inverted_index_2.pkl', 'rb') as f:
    inverted_index_2 = pickle.load(f)

##### An example of the tfidf values of the term 'american'

In [24]:
print(inverted_index_2[22851])

{16: 1.0693743579103792, 22: 1.0693743579103792, 42: 2.1387487158207583, 50: 1.0693743579103792, 56: 3.2081230737311373, 59: 1.0693743579103792, 77: 2.1387487158207583, 96: 1.0693743579103792, 105: 1.0693743579103792, 106: 1.0693743579103792, 107: 1.0693743579103792, 110: 1.0693743579103792, 123: 1.0693743579103792, 129: 1.0693743579103792, 130: 1.0693743579103792, 141: 3.2081230737311373, 149: 1.0693743579103792, 157: 3.2081230737311373, 164: 1.0693743579103792, 173: 1.0693743579103792, 174: 1.0693743579103792, 183: 1.0693743579103792, 221: 1.0693743579103792, 224: 2.1387487158207583, 225: 1.0693743579103792, 277: 1.0693743579103792, 287: 3.2081230737311373, 320: 2.1387487158207583, 329: 1.0693743579103792, 335: 2.1387487158207583, 338: 1.0693743579103792, 341: 1.0693743579103792, 353: 1.0693743579103792, 356: 1.0693743579103792, 360: 1.0693743579103792, 369: 1.0693743579103792, 382: 1.0693743579103792, 400: 1.0693743579103792, 412: 1.0693743579103792, 415: 1.0693743579103792, 425: 1.

#### Now we can store the norm of a single document as a dataframe column because is not depend on the query given.

In [25]:
df_cleaned.head(5)

,placeName,placeCleanDesc,placeURL
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",http://www.atlasobscura.com/places/catacombes-...


In [26]:
import numpy as np

def get_norma(dataframe):
    list_norm = []
    norma = {}
    for doc in tqdm(range(len(dataframe.index))):
        all_tfidf_doc = []
        for word in dataframe.placeCleanDesc.loc[doc]:
            term_id = voc[word]
            tfidf_term_id = inverted_index_2[term_id][doc]
            all_tfidf_doc.append(tfidf_term_id)
        list_norm = np.linalg.norm(all_tfidf_doc)
        norma[doc] = list_norm
    
    return norma

In [27]:
df_cleaned['norma'] = [value for key, value in get_norma(df_cleaned).items()]
df_cleaned

100%|██████████| 7200/7200 [00:01<00:00, 4542.48it/s]


,placeName,placeCleanDesc,placeURL,norma
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",http://www.atlasobscura.com/places/city-hall-s...,52.138533
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",http://www.atlasobscura.com/places/highgate-ce...,51.845016
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",http://www.atlasobscura.com/places/leadenhall-...,16.494082
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",http://www.atlasobscura.com/places/wave-organ,24.926279
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",http://www.atlasobscura.com/places/catacombes-...,41.629849
...,...,...,...,...
7195,Athenry Dominican Priory,"[locat, east, galway, citi, small, town, athen...",http://www.atlasobscura.com/places/athenry-dom...,29.964222
7196,The Cairo,"[washington, residenti, neighborhood, typifi, ...",http://www.atlasobscura.com/places/the-cairo,59.252483
7197,Winganon Space Capsule,"[drive, along, dull, stretch, road, talala, wi...",http://www.atlasobscura.com/places/winganon-sp...,15.477862
7198,Cook & Book,"[cook, book, brussel, far, bookstor, size, sup...",http://www.atlasobscura.com/places/cook-book,12.384734


### 2.2.2) Execute the query

In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the tfIdf representations of the documents.

Given a query input by the user, for example: **american museum**


The search engine is supposed to return a list of documents, ranked by their Cosine Similarity to the query entered in the input.

More precisely, the output must contain:

1. placeName
1. placeDesc
1. placeURL

The similarity score of the documents with respect to the query (float value between 0 and 1)

### We have to store the tfidf of the query to can calculate the cosine similarity 

In [28]:
def get_tf(query):
    
    tf_query = {}

    for token in match_query(query):
        if token not in list(tf_query.keys()):
            tf_query[token] = 0
        tf_query[token] += 1

    return tf_query

In [29]:
tf_query = get_tf(query)
tf_query

{22851: 1, 21370: 1}

In [30]:
def get_tfidf(query, dataframe):
    
    tfidf_query = {}

    for key in get_tf(query).keys():
        tfidf_query[key] = get_tf(query)[key]/np.log10(len(dataframe.placeCleanDesc)/len(inverted_index_2[key]))
    
    return tfidf_query

In [31]:
tfidf_query = get_tfidf(query, df_cleaned)
tfidf_query

{22851: 1.0693743579103792, 21370: 1.4591857990867851}

#### Cosine Similarity

In [32]:
def cosinesim(query, inverted_index_2, dataframe):
    
    list_num = {}
    for doc in search_engine(query):
        num_sim = 0.0
        for term_id in get_tfidf(query, dataframe).keys():
            num_sim += (get_tfidf(query, dataframe)[term_id]) * (inverted_index_2[term_id][doc])
        sim = num_sim/dataframe.norma.loc[doc]
        list_num[doc] = sim
    
    return list_num

In [33]:
similarity = cosinesim(query, inverted_index_2, df_cleaned)
similarity

{2052: 0.39633598356691063,
 1541: 0.2850603449576355,
 6154: 0.15415066057083565,
 2571: 0.11732251299521528,
 3082: 0.550682204365452,
 4621: 0.22873578265173064,
 1039: 0.38465396359749243,
 3088: 0.062138436980614865,
 1551: 0.2153262326232052,
 1552: 0.07823220907963695,
 1554: 0.41531482737066067,
 2580: 0.036002001159498496,
 6680: 0.12492458008610073,
 1050: 0.07251001389807321,
 3611: 0.3923160061510483,
 5148: 0.2801058417529959,
 2077: 0.4295527104392878,
 2081: 0.4160410614049794,
 2088: 0.13145157694338153,
 6697: 0.10711027366423023,
 1580: 0.056133377013180746,
 5678: 0.48116399509356306,
 2607: 0.17667156297481051,
 5167: 0.34575859756876154,
 1072: 0.20733778084412305,
 6195: 0.11873915170849583,
 1589: 0.39450252239147016,
 6198: 0.3113748285008601,
 2105: 0.4378227230628263,
 59: 0.13340616796406293,
 5179: 0.08122639438784653,
 6205: 0.41552085326819393,
 1086: 0.17608672893247054,
 4673: 0.10071403034446207,
 6210: 0.3461196666343699,
 6211: 0.08869955044066134,
 1

In [34]:
print(max(similarity.values()))    

0.6847530464707337


#### Now, we want the $top-k$ documents that are most similar to the query given in input.

#### For this purpose, we use the $heap-structure$

##### Heaps are arrays for which a[k] <= a[2*k+1] and a[k] <= a[2*k+2] for all k, counting elements from 0.  For the sake of comparison, non-existing elements are considered to be infinite.  The interesting property of a heap is that a[0] is always its smallest element.

##### Remind that the  query  is  "american museum"

### Second Search Engine

In [35]:
import heapq
from heapq import heappop, heapify 

def search_engine_2(query, k):
    similarity = cosinesim(query, inverted_index_2, df_cleaned)
    #case_1: all the documents with non-zero similarity with the query when the results are less than k
    if len(search_engine(query)) < k:
        docs_sim = enumerate(-similarity)
        docs_sim = [item[::-1] for item in docs_sim]
        heapq.heapify(docs_sim)
        k = len(np.nonzero(similarity))
    else: 
    #case_2: at least k documents most similar to the query in input
        docs_sim = [(-similarity[doc], doc) for doc in search_engine(query)]
        heapq.heapify(docs_sim)
    
    #get top k documents
    top_k_documents = []
    for i in range(k):
        top_k_documents.append(heapq.heappop(docs_sim))
    
    return top_k_documents

In [36]:
def display_top_k_documents(query, k):
    
    top_k_documents = search_engine_2(query, k)
    
    indexes = [doc[1] for doc in top_k_documents]
    col = ['placeName', 'placeDesc', 'placeURL', 'similarity']
    output = pd.DataFrame(columns= col, index = indexes)
    
    output.placeName = [df_places.placeName[doc[1]] for doc in top_k_documents]
    output.placeDesc = [df_places.placeDesc[doc[1]] for doc in top_k_documents]
    output.placeURL = [df_places.placeURL[doc[1]] for doc in top_k_documents]
    output.similarity = [-doc[0] for doc in top_k_documents]
    
    return output

### These are the top 10 places in which the query is (change k if you want)

In [39]:
display_top_k_documents(query, 10)

,placeName,placeDesc,placeURL,similarity
4750,Mount Angel Abbey Museum,The Mount Angel Abbey Museum is a very eclecti...,http://www.atlasobscura.com/places/mount-angel...,0.684753
4465,American Writers Museum,The American Writers Museum—tucked away on the...,http://www.atlasobscura.com/places/american-wr...,0.664380
4459,Miracle of America Museum,"Moonshine stills, an 1880s hearse, UFOs, pro- ...",http://www.atlasobscura.com/places/miracle-of-...,0.624144
5014,Mammoth Cave Wildlife Museum,Cave City is a wonderland of kitschy tourist t...,http://www.atlasobscura.com/places/mammoth-cav...,0.566252
3082,Museum of Russian Culture,"When you think San Francisco, you probably don...",http://www.atlasobscura.com/places/museum-of-r...,0.550682
6250,American Banjo Museum,Containing more instruments on public display ...,http://www.atlasobscura.com/places/american-ba...,0.548884
3958,Tamástslikt Cultural Institute,"The Tamástslikt Cultural Institute, situated o...",http://www.atlasobscura.com/places/tamastslikt...,0.546977
6644,Evel Knievel Museum,The Evel Knievel Museum takes you through the ...,http://www.atlasobscura.com/places/evel-knieve...,0.540213
3409,Geppi's Entertainment Museum,It’s a unique place that can create a sentimen...,http://www.atlasobscura.com/places/geppi-s-ent...,0.533605
6016,Isaac Hayes's Gold-Plated Cadillac,Stax Museum of American Soul Music celebrates ...,http://www.atlasobscura.com/places/isaac-hayes...,0.533066


# 3. Define a new score!
Now it's your turn. Build a new metric to rank places based on the queries of their users.

In this scenario, a single user can give input more information than a single textual query, so you need to consider all this information and think of a creative and logical way to answer the user's requests.

Practically:

The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 3.1.

Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the plot of the documents; you must use the remaining variables in your dataset (or new possible variables that you can create from the existing ones). You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

Q: How to sort them? A: Allow the user to specify more information that you find in the documents and define a new metric that ranks the results based on the new request. You can also use other information regarding the place to score some places above others.

N.B.: You have to define a scoring function, not a filter!

The output, must contain:

placeName
placeDesc
placeURL
The new similarity score of the documents with respect to the query
Are the results you obtain better than with the previous scoring function? Explain and compare results.

In [40]:
df_places.head(5)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1828,8583,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,"['rebekah-otto', 'jrc2692']",2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...
1,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2618,8182,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,['ctlssnghts'],2014-08-09,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...
2,Leadenhall Market,"['markets', 'film-locations', 'film']",3135,7570,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,"['meghanneal', 'julius-spada']",2016-08-01,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...
3,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...",2429,7408,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,"['mbison', 'mark-casey']",2008-11-21,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",http://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...",4447,7046,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,"['cpilgrim', 'jarednagel']",2009-02-13,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",http://www.atlasobscura.com/places/catacombes-...


In [41]:
df_cleaned.head(5)

,placeName,placeCleanDesc,placeURL,norma
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",http://www.atlasobscura.com/places/city-hall-s...,52.138533
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",http://www.atlasobscura.com/places/highgate-ce...,51.845016
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",http://www.atlasobscura.com/places/leadenhall-...,16.494082
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",http://www.atlasobscura.com/places/wave-organ,24.926279
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",http://www.atlasobscura.com/places/catacombes-...,41.629849


### Take all information we need

In [42]:
df_cleaned_2 = pd.DataFrame(columns = [["placeName", "placeCleanDesc", "numPeopleVisited",
                                        "numPeopleWant", "placeAddress", "placeURL", "norma"]])
df_cleaned_2.placeName = df_places.placeName
df_cleaned_2.placeCleanDesc = df_cleaned.placeCleanDesc
df_cleaned_2.numPeopleVisited = df_places.numPeopleVisited
df_cleaned_2.numPeopleWant = df_places.numPeopleWant
df_cleaned_2.placeAddress = df_places.placeAddress
df_cleaned_2.placeURL = df_places.placeURL
df_cleaned_2.norma = df_cleaned.norma

df_cleaned_2

,placeName,placeCleanDesc,numPeopleVisited,numPeopleWant,placeAddress,placeURL,norma
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",1828,8583,"31 Centre St New York, New York, 10007 United ...",http://www.atlasobscura.com/places/city-hall-s...,52.138533
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",2618,8182,"Swain's Lane, Highgate London, England, N6 Uni...",http://www.atlasobscura.com/places/highgate-ce...,51.845016
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",3135,7570,"London, England, EC3V United Kingdom",http://www.atlasobscura.com/places/leadenhall-...,16.494082
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",2429,7408,"83 Marina Green Dr San Francisco, California, ...",http://www.atlasobscura.com/places/wave-organ,24.926279
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",4447,7046,"1 Place Denfert-Rochereau Paris, 75014 France",http://www.atlasobscura.com/places/catacombes-...,41.629849
...,...,...,...,...,...,...,...
7195,Athenry Dominican Priory,"[locat, east, galway, citi, small, town, athen...",86,406,1 Abbey Row Athenry Ireland,http://www.atlasobscura.com/places/athenry-dom...,29.964222
7196,The Cairo,"[washington, residenti, neighborhood, typifi, ...",342,405,"1615 Q Street NW Washington, District of Colum...",http://www.atlasobscura.com/places/the-cairo,59.252483
7197,Winganon Space Capsule,"[drive, along, dull, stretch, road, talala, wi...",120,405,"E 300 Road Talala, Oklahoma United States",http://www.atlasobscura.com/places/winganon-sp...,15.477862
7198,Cook & Book,"[cook, book, brussel, far, bookstor, size, sup...",124,405,1 Place du Temps Libre Woluwe-Saint-Lambert Br...,http://www.atlasobscura.com/places/cook-book,12.384734


### For this purpose, let's imagine that a user wants to go to a place in a certain country.

#### In this case we will do the search for these specific countries: 
1. Italy 
1. France
1. Spain
1. England

#### What are best interesting places similar to his request?

#### Take all states together as a first request, then we'll do our second request based on which type of place we want to visit

In [46]:
states = input(list).split(',')

<class 'list'>Italy,France,Spain,England


In [48]:
states

['Italy', 'France', 'Spain', 'England']

#### Modify 'placeAddress' column to pick only places related on these four states of our interest.

In [49]:
docs_addr = {}
for row in tqdm(range(7200)):
    addr = (',').join(df_cleaned_2.placeAddress.loc[row]).split(',')
    new_addr = (',').join(addr).split(' ')
    docs_addr[row] = new_addr

100%|██████████| 7200/7200 [00:08<00:00, 869.73it/s]


In [50]:
print(docs_addr)

{0: ['31', 'Centre', 'St', 'New', 'York,', 'New', 'York,', '10007', 'United', 'States'], 1: ["Swain's", 'Lane,', 'Highgate', 'London,', 'England,', 'N6', 'United', 'Kingdom'], 2: ['London,', 'England,', 'EC3V', 'United', 'Kingdom'], 3: ['83', 'Marina', 'Green', 'Dr', 'San', 'Francisco,', 'California,', '94123', 'United', 'States'], 4: ['1', 'Place', 'Denfert-Rochereau', 'Paris,', '75014', 'France'], 5: ['687', 'Broadway', 'New', 'York,', 'New', 'York,', '10012', 'United', 'States'], 6: ['', 'Pier', '39,', 'Building', 'O-11', 'San', 'Francisco,', 'California', 'United', 'States'], 7: ['320', 'W', 'Temple', 'St', 'Los', 'Angeles,', 'California,', '90012', 'United', 'States'], 8: ['1700', '16th', 'Avenue', 'San', 'Francisco,', 'California,', '94122', 'United', 'States'], 9: ['275', 'Church', 'St', 'New', 'York,', 'New', 'York,', '10013', 'United', 'States'], 10: ['6031', 'Hollywood', 'Blvd.', 'Los', 'Angeles,', 'California,', '90028', 'United', 'States'], 11: ['453', 'S.', 'Spring', 'St.'

In [51]:
df_cleaned_2['placeAddress'] = docs_addr.values()
df_cleaned_2

,placeName,placeCleanDesc,numPeopleVisited,numPeopleWant,placeAddress,placeURL,norma
0,City Hall Station,"[first, new, york, citi, subway, built, oper, ...",1828,8583,"[31, Centre, St, New, York,, New, York,, 10007...",http://www.atlasobscura.com/places/city-hall-s...,52.138533
1,Highgate Cemetery,"[open, 1839, highgat, one, london, infam, ceme...",2618,8182,"[Swain's, Lane,, Highgate, London,, England,, ...",http://www.atlasobscura.com/places/highgate-ce...,51.845016
2,Leadenhall Market,"[ornat, paint, roof, cobbl, floor, leadenhal, ...",3135,7570,"[London,, England,, EC3V, United, Kingdom]",http://www.atlasobscura.com/places/leadenhall-...,16.494082
3,The Wave Organ,"[locat, jetti, san, francisco, bay, wave, orga...",2429,7408,"[83, Marina, Green, Dr, San, Francisco,, Calif...",http://www.atlasobscura.com/places/wave-organ,24.926279
4,Catacombes de Paris,"[2004, parisian, polic, assign, train, exercis...",4447,7046,"[1, Place, Denfert-Rochereau, Paris,, 75014, F...",http://www.atlasobscura.com/places/catacombes-...,41.629849
...,...,...,...,...,...,...,...
7195,Athenry Dominican Priory,"[locat, east, galway, citi, small, town, athen...",86,406,"[1, Abbey, Row, Athenry, Ireland]",http://www.atlasobscura.com/places/athenry-dom...,29.964222
7196,The Cairo,"[washington, residenti, neighborhood, typifi, ...",342,405,"[1615, Q, Street, NW, Washington,, District, o...",http://www.atlasobscura.com/places/the-cairo,59.252483
7197,Winganon Space Capsule,"[drive, along, dull, stretch, road, talala, wi...",120,405,"[E, 300, Road, Talala,, Oklahoma, United, States]",http://www.atlasobscura.com/places/winganon-sp...,15.477862
7198,Cook & Book,"[cook, book, brussel, far, bookstor, size, sup...",124,405,"[1, Place, du, Temps, Libre, Woluwe-Saint-Lamb...",http://www.atlasobscura.com/places/cook-book,12.384734


#### What are the correct places based on the state's request?

In [60]:
correct_docs = []

for state in states:
    docs = [doc for doc, row in docs_addr.items() if state in row]
    correct_docs.append(docs)

correct_docs

[[81,
  91,
  131,
  150,
  216,
  245,
  289,
  321,
  397,
  402,
  421,
  472,
  480,
  572,
  609,
  631,
  675,
  684,
  761,
  797,
  829,
  839,
  841,
  849,
  885,
  897,
  904,
  940,
  965,
  966,
  971,
  986,
  1010,
  1051,
  1118,
  1130,
  1261,
  1307,
  1384,
  1394,
  1430,
  1475,
  1484,
  1489,
  1498,
  1525,
  1571,
  1592,
  1600,
  1712,
  1794,
  1809,
  1820,
  1859,
  1884,
  1908,
  2004,
  2048,
  2134,
  2145,
  2207,
  2299,
  2315,
  2386,
  2449,
  2452,
  2457,
  2471,
  2507,
  2532,
  2542,
  2546,
  2615,
  2703,
  2718,
  2730,
  2838,
  2897,
  2913,
  2930,
  2933,
  2963,
  3061,
  3076,
  3099,
  3116,
  3117,
  3128,
  3162,
  3210,
  3220,
  3226,
  3354,
  3387,
  3406,
  3471,
  3500,
  3550,
  3608,
  3645,
  3733,
  3825,
  3956,
  3966,
  3967,
  3989,
  4030,
  4088,
  4109,
  4129,
  4144,
  4151,
  4156,
  4182,
  4204,
  4210,
  4222,
  4248,
  4255,
  4321,
  4367,
  4383,
  4386,
  4428,
  4435,
  4469,
  4473,
  4492,
  4501,
  

### List of all places of these four states in the request

In [62]:
all_places = [item for items in correct_docs for item in items]
print(len(all_places))
all_places

630


[81,
 91,
 131,
 150,
 216,
 245,
 289,
 321,
 397,
 402,
 421,
 472,
 480,
 572,
 609,
 631,
 675,
 684,
 761,
 797,
 829,
 839,
 841,
 849,
 885,
 897,
 904,
 940,
 965,
 966,
 971,
 986,
 1010,
 1051,
 1118,
 1130,
 1261,
 1307,
 1384,
 1394,
 1430,
 1475,
 1484,
 1489,
 1498,
 1525,
 1571,
 1592,
 1600,
 1712,
 1794,
 1809,
 1820,
 1859,
 1884,
 1908,
 2004,
 2048,
 2134,
 2145,
 2207,
 2299,
 2315,
 2386,
 2449,
 2452,
 2457,
 2471,
 2507,
 2532,
 2542,
 2546,
 2615,
 2703,
 2718,
 2730,
 2838,
 2897,
 2913,
 2930,
 2933,
 2963,
 3061,
 3076,
 3099,
 3116,
 3117,
 3128,
 3162,
 3210,
 3220,
 3226,
 3354,
 3387,
 3406,
 3471,
 3500,
 3550,
 3608,
 3645,
 3733,
 3825,
 3956,
 3966,
 3967,
 3989,
 4030,
 4088,
 4109,
 4129,
 4144,
 4151,
 4156,
 4182,
 4204,
 4210,
 4222,
 4248,
 4255,
 4321,
 4367,
 4383,
 4386,
 4428,
 4435,
 4469,
 4473,
 4492,
 4501,
 4537,
 4650,
 4653,
 4667,
 4698,
 4795,
 4813,
 4814,
 4855,
 4866,
 4871,
 4873,
 4946,
 5013,
 5020,
 5029,
 5037,
 5079,
 5089

### Our new dataset!

In [223]:
df_new = df_cleaned_2.loc[all_places]
df_new

,placeName,placeCleanDesc,numPeopleVisited,numPeopleWant,placeAddress,placeURL,norma
81,Santa Maria della Concezione Crypts,"[1775, marqui, de, sade, wrote, never, seen, a...",2065,3523,"[Via, Vittorio, Veneto,, 27, Rome,, 00187, Italy]",http://www.atlasobscura.com/places/santa-maria...,26.337529
91,Valle dei Mulini (Valley of the Mills),"[vall, dei, mulini, valley, mill, verdant, gro...",467,3410,"[Viale, Enrico, Caruso, Sorrento,, 80067, Italy]",http://www.atlasobscura.com/places/valle-dei-m...,26.273220
131,Torre Argentina (Roman Cat Sanctuary),"[rome, cat, ancient, known, largo, di, torr, a...",2497,3023,"[1-41, Via, di, Torre, Argentina, Rome,, 00186...",http://www.atlasobscura.com/places/torre-argen...,22.396246
150,The Monsters of Bomarzo,"[park, monster, parco, dei, mostri, garden, bo...",509,2881,"[101, Via, Madonna, della, Valle, Bomarzo,, 01...",http://www.atlasobscura.com/places/the-monster...,41.710853
216,Libreria Acqua Alta,"[keep, collect, book, citi, road, made, water,...",1183,2577,"[5173, Calle, Lunga, 5176/B, Venice, Italy]",http://www.atlasobscura.com/places/libreria-ac...,18.785860
...,...,...,...,...,...,...,...
6936,Richmond Park,"[1625, particularli, virul, outbreak, bubon, p...",561,420,"[Unnamed, Road, London,, England, United, King...",http://www.atlasobscura.com/places/richmond-de...,24.750489
7083,'Coventry Doom',"[holi, triniti, church, center, coventri, engl...",67,411,"[5A, Priory, Row, Coventry,, England, United, ...",http://www.atlasobscura.com/places/coventry-doom,18.586080
7108,Northumberlandia: Lady of the North,"[walk, four, mile, swirl, path, wind, around, ...",156,409,"[1, Blagdon, Lane, Northumberland,, England, U...",http://www.atlasobscura.com/places/northuberla...,15.076414
7153,One Tree Hill,"[surviv, fragment, much, larger, great, north,...",107,407,"[69, Honor, Oak, Rise, London,, England, Unite...",http://www.atlasobscura.com/places/one-tree-hill,40.339883


### Define new targets changing values on the column of our dataset useful for our new score.

#### We normalized the amount of people visited and want to go in that place and the weight of each documents

In [224]:
def new_targets(dataframe):

    #averages
    avg_visit = dataframe.numPeopleVisited.mean()
    avg_want = dataframe.numPeopleWant.mean()
    
    #normallization
    dataframe['numPeopleVisited'] = dataframe['numPeopleVisited'].div(avg_visit)
    dataframe['numPeopleWant'] = dataframe['numPeopleWant'].div(avg_want)
    
    #final score will be a combination of these measures above with weights(degrees of belief) given by us
    dataframe['numPeopleVisited'] = 0.3*dataframe['numPeopleVisited']
    dataframe['numPeopleWant'] = 0.7*dataframe['numPeopleWant']
    dataframe['norma'] = 1/dataframe['norma']
    
    return (dataframe)

In [225]:
df_new = new_targets(df_new)
df_new

,placeName,placeCleanDesc,numPeopleVisited,numPeopleWant,placeAddress,placeURL,norma
81,Santa Maria della Concezione Crypts,"[1775, marqui, de, sade, wrote, never, seen, a...",1.503733,2.740029,"[Via, Vittorio, Veneto,, 27, Rome,, 00187, Italy]",http://www.atlasobscura.com/places/santa-maria...,0.037969
91,Valle dei Mulini (Valley of the Mills),"[vall, dei, mulini, valley, mill, verdant, gro...",0.340070,2.652143,"[Viale, Enrico, Caruso, Sorrento,, 80067, Italy]",http://www.atlasobscura.com/places/valle-dei-m...,0.038062
131,Torre Argentina (Roman Cat Sanctuary),"[rome, cat, ancient, known, largo, di, torr, a...",1.818316,2.351152,"[1-41, Via, di, Torre, Argentina, Rome,, 00186...",http://www.atlasobscura.com/places/torre-argen...,0.044650
150,The Monsters of Bomarzo,"[park, monster, parco, dei, mostri, garden, bo...",0.370654,2.240711,"[101, Via, Madonna, della, Valle, Bomarzo,, 01...",http://www.atlasobscura.com/places/the-monster...,0.023975
216,Libreria Acqua Alta,"[keep, collect, book, citi, road, made, water,...",0.861461,2.004273,"[5173, Calle, Lunga, 5176/B, Venice, Italy]",http://www.atlasobscura.com/places/libreria-ac...,0.053232
...,...,...,...,...,...,...,...
6936,Richmond Park,"[1625, particularli, virul, outbreak, bubon, p...",0.408520,0.326657,"[Unnamed, Road, London,, England, United, King...",http://www.atlasobscura.com/places/richmond-de...,0.040403
7083,'Coventry Doom',"[holi, triniti, church, center, coventri, engl...",0.048789,0.319657,"[5A, Priory, Row, Coventry,, England, United, ...",http://www.atlasobscura.com/places/coventry-doom,0.053804
7108,Northumberlandia: Lady of the North,"[walk, four, mile, swirl, path, wind, around, ...",0.113599,0.318102,"[1, Blagdon, Lane, Northumberland,, England, U...",http://www.atlasobscura.com/places/northuberla...,0.066329
7153,One Tree Hill,"[surviv, fragment, much, larger, great, north,...",0.077917,0.316546,"[69, Honor, Oak, Rise, London,, England, Unite...",http://www.atlasobscura.com/places/one-tree-hill,0.024789


#### Our new score it will be the interest rate of all people based on how many people want to visit the place and how many people would like to go to it in average in that country

In [226]:
def score_function(dataframe):
    
    df_new['score'] = df_new.iloc[:, 2:4].sum(axis=1)/2
    final_data = pd.DataFrame(dataframe)
    final_data['score'] = (final_data.iloc[:, 6:8].sum(axis=1))
    
    return(final_data)

In [227]:
final_data = score_function(df_new)
final_data

,placeName,placeCleanDesc,numPeopleVisited,numPeopleWant,placeAddress,placeURL,norma,score
81,Santa Maria della Concezione Crypts,"[1775, marqui, de, sade, wrote, never, seen, a...",1.503733,2.740029,"[Via, Vittorio, Veneto,, 27, Rome,, 00187, Italy]",http://www.atlasobscura.com/places/santa-maria...,0.037969,2.159850
91,Valle dei Mulini (Valley of the Mills),"[vall, dei, mulini, valley, mill, verdant, gro...",0.340070,2.652143,"[Viale, Enrico, Caruso, Sorrento,, 80067, Italy]",http://www.atlasobscura.com/places/valle-dei-m...,0.038062,1.534168
131,Torre Argentina (Roman Cat Sanctuary),"[rome, cat, ancient, known, largo, di, torr, a...",1.818316,2.351152,"[1-41, Via, di, Torre, Argentina, Rome,, 00186...",http://www.atlasobscura.com/places/torre-argen...,0.044650,2.129384
150,The Monsters of Bomarzo,"[park, monster, parco, dei, mostri, garden, bo...",0.370654,2.240711,"[101, Via, Madonna, della, Valle, Bomarzo,, 01...",http://www.atlasobscura.com/places/the-monster...,0.023975,1.329657
216,Libreria Acqua Alta,"[keep, collect, book, citi, road, made, water,...",0.861461,2.004273,"[5173, Calle, Lunga, 5176/B, Venice, Italy]",http://www.atlasobscura.com/places/libreria-ac...,0.053232,1.486099
...,...,...,...,...,...,...,...,...
6936,Richmond Park,"[1625, particularli, virul, outbreak, bubon, p...",0.408520,0.326657,"[Unnamed, Road, London,, England, United, King...",http://www.atlasobscura.com/places/richmond-de...,0.040403,0.407992
7083,'Coventry Doom',"[holi, triniti, church, center, coventri, engl...",0.048789,0.319657,"[5A, Priory, Row, Coventry,, England, United, ...",http://www.atlasobscura.com/places/coventry-doom,0.053804,0.238027
7108,Northumberlandia: Lady of the North,"[walk, four, mile, swirl, path, wind, around, ...",0.113599,0.318102,"[1, Blagdon, Lane, Northumberland,, England, U...",http://www.atlasobscura.com/places/northuberla...,0.066329,0.282179
7153,One Tree Hill,"[surviv, fragment, much, larger, great, north,...",0.077917,0.316546,"[69, Honor, Oak, Rise, London,, England, Unite...",http://www.atlasobscura.com/places/one-tree-hill,0.024789,0.222021


### Now is given a place, which will be the most interesting?

### We have chosen to go in the best park of these states, but we want have a choice, so we require the top ten parks in these set of places!

In [64]:
new_query = input(str)

<class 'str'>park


In [65]:
cleaned_query_2 = clean_query(new_query)
cleaned_query_2

['park']

In [66]:
map_query_2 = match_query(new_query)
map_query_2

[25794]

In [67]:
tf_query_2 = get_tf(new_query)
tf_query_2

{25794: 1}

In [68]:
tfidf_query_2 = get_tfidf(new_query, df_cleaned_2)
tfidf_query_2

{25794: 1.4604305857256918}

### Which of all these places are a park? Make a search engine 

In [212]:
match_results_2 = (search_engine(new_query)).intersection(set(all_places))
match_results_2

{80,
 150,
 321,
 363,
 402,
 547,
 621,
 640,
 797,
 839,
 897,
 930,
 1118,
 1384,
 1430,
 1549,
 1604,
 1613,
 1736,
 1758,
 2048,
 2224,
 2392,
 2398,
 2659,
 2836,
 2943,
 3556,
 3608,
 3791,
 3882,
 3956,
 4088,
 4146,
 4364,
 4383,
 4386,
 4501,
 4516,
 4667,
 4813,
 4814,
 4855,
 5057,
 5156,
 5200,
 5555,
 5603,
 5662,
 5755,
 5784,
 5811,
 5845,
 5941,
 6094,
 6119,
 6282,
 6450,
 6480,
 6493,
 6683,
 6723,
 6936,
 7023,
 7108,
 7130,
 7153}

#### All new scores of all places(park in this case) in the four states we have considered.

In [228]:
scores = []
for row in match_results_2:
    sim = {row: scores for scores in final_data.score.loc[row]}
    scores.append(sim)

similarity = {}
for d in scores:
    similarity.update(d)
similarity

{2048: 0.42288476049218965,
 640: 0.9268698139308091,
 897: 0.6844114522560744,
 6282: 0.3179500762347848,
 4364: 0.36862841621454556,
 1549: 0.9426219011845327,
 402: 1.078591988957368,
 2836: 0.3641686293300595,
 4501: 0.28783248790065846,
 150: 1.3296568316350725,
 1430: 0.5368400081844149,
 6936: 0.4079918427307492,
 3608: 0.3213224035080904,
 5784: 0.28410842967592354,
 6683: 0.2200192357418888,
 797: 0.6870513228434486,
 5662: 0.2899421932882373,
 4383: 0.31096052760099746,
 4386: 0.38234233074003326,
 547: 0.9990226683721923,
 4516: 0.2772655343416835,
 930: 0.8317542403127838,
 5156: 0.6252786783262094,
 3882: 0.38313731988726746,
 2224: 0.665730600249737,
 4146: 0.4879761247566045,
 6450: 0.3290766451641652,
 5555: 0.3033027560933172,
 5811: 0.22581260436906997,
 5941: 0.388486143910042,
 4667: 0.283341802181216,
 321: 0.9460166155044377,
 5057: 0.3142953706164235,
 6723: 0.2509379434046206,
 7108: 0.2821791739763978,
 1604: 1.2487931786353437,
 839: 1.2668214993267572,
 1736:

### Third Search Engine always with heap-structure

In [229]:
import heapq
from heapq import heappop, heapify 

def search_engine_3(k):

    #case_1: all the documents with non-zero similarity with the query when the results are less than k
    if len(match_results_2) < k:
            docs_sim = enumerate(-similarity)
            docs_sim = [item[::-1] for item in docs_sim]
            heapq.heapify(docs_sim)
            k = len(np.nonzero(similarity))
    #case_2: at least k documents most similar to the query in input
    else:
        docs_sim = [(-similarity[doc], doc) for doc in match_results_2]
        heapq.heapify(docs_sim)
    
    #get top k documents
    top_k_documents = []
    for i in range(k):
        top_k_documents.append(heapq.heappop(docs_sim))
    
    return top_k_documents

In [230]:
documents = search_engine_3(10)
documents

[(-2.1673259976554644, 621),
 (-1.6770969359728065, 80),
 (-1.3296568316350725, 150),
 (-1.2668214993267572, 839),
 (-1.2487931786353437, 1604),
 (-1.078591988957368, 402),
 (-1.0668368551608343, 363),
 (-0.9990226683721923, 547),
 (-0.9460166155044377, 321),
 (-0.9426219011845327, 1549)]

In [236]:
def display_top_k_documents(k):
    
    top_k_documents = search_engine_3(k)
    
    indexes = [doc[1] for doc in top_k_documents]
    col = ['placeName', 'placeDesc', 'placeAddress', 'placeAlt', 'placeLong', 'placeURL', 'score']
    output = pd.DataFrame(columns= col, index = indexes)
    
    output.placeName = [df_places.placeName[doc[1]] for doc in top_k_documents]
    output.placeDesc = [df_places.placeDesc[doc[1]] for doc in top_k_documents]
    output.placeURL = [df_places.placeURL[doc[1]] for doc in top_k_documents]
    output.score = [-doc[0] for doc in top_k_documents]
    output.placeAlt = [df_places.placeAlt[doc[1]] for doc in top_k_documents]
    output.placeLong = [df_places.placeLong[doc[1]] for doc in top_k_documents]
    output.placeAddress = [df_places.placeAddress[doc[1]] for doc in top_k_documents]
    
    return output

In [237]:
our_output = display_top_k_documents(10)
our_output

,placeName,placeDesc,placeAddress,placeAlt,placeLong,placeURL,score
621,Park Güell,"At Park Güell, stone, tile, plants, and Medite...","s/n Carrer d'Olot Barcelona, 08024 Spain",41.4136,2.1528,http://www.atlasobscura.com/places/park-guell,2.167326
80,Petite Ceinture,The Petite Ceinture railway circling through t...,"jean moulin station Paris, 75015 France",48.8350,2.2851,http://www.atlasobscura.com/places/petite-cein...,1.677097
150,The Monsters of Bomarzo,"The Park of the Monsters, or “Parco dei Mostri...","101 Via Madonna della Valle Bomarzo, 01020 Italy",42.4910,12.2462,http://www.atlasobscura.com/places/the-monster...,1.329657
839,Via Appia,"Walking from the center of Rome, Via Appia, or...","Via Appia Antica, 42 Rome, 00147 Italy",41.8675,12.5028,http://www.atlasobscura.com/places/via-appia-a...,1.266821
1604,Speakers' Corner,Free speech laws in the United Kingdom include...,"Hyde Park London, England United Kingdom",51.5120,-0.1592,http://www.atlasobscura.com/places/speakers-co...,1.248793
402,Porta Alchemica,"While this “magic door” is famous to Romans, i...","Piazza Vittorio Emanuele In the park Rome, 001...",41.8951,12.5037,http://www.atlasobscura.com/places/porta-magica,1.078592
363,Labyrinth Park of Horta,The archetypical hedge maze sitting in the cen...,"Passeig dels Castanyers, 1 Barcelona, 08035 Spain",41.4379,2.1476,http://www.atlasobscura.com/places/labyrinth-p...,1.066837
547,Kyoto Garden,Among the many lovely features of west London’...,"Holland Park Ave London, England United Kingdom",51.5023,-0.2043,http://www.atlasobscura.com/places/kyoto-garde...,0.999023
321,The Sunken City of Baia,"In its heyday, the Classical Roman city of Bai...","via Napoli 1 Baia, 80070 Italy",40.8184,14.0769,http://www.atlasobscura.com/places/the-sunken-...,0.946017
1549,Palacio de Cristal,"Built to mirror a similar structure in London,...","Plaza de la Independencia, 7 Madrid, 28009 Spain",40.4136,-3.6821,http://www.atlasobscura.com/places/palacio-de-...,0.942622


# 4. Visualizing the most relevant places
Using maps can help people understand how far one place is from another so they can plan their trips more adequately. Here we challenge you to show a map with the places found with the score defined in point 3. Ensure you can at least identify and visualize the name, city, country, address and the number of people who visited each place. You can find some ideas on how to create maps in Python here and here but don't limit yourselves, let your minds fly!!

In [256]:
import plotly.express as px

fig=px.scatter_mapbox(our_output, title = 'Place Atlas', lat = "placeAlt", lon = "placeLong", hover_name = "placeName", color = "placeAddress",
                      color_continuous_scale = px.colors.sequential.Viridis, size="score", zoom=3.5, height=500)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update(layout={'title': {'x': 0.4, 'y': 0.8}})
fig.show()

# 5. BONUS: More complex search engine
IMPORTANT: This is a bonus step, so it's not mandatory. You can get the maximum score also without doing this. We will take this into account, only if the rest of the homework has been completed.

For the Bonus part, we want to ask you more sophisticated search engine. Here we want to let users issue more complex queries. The options of this new search engine are:

Give the possibility to specify queries for the following features (the user should have the option to issue none or all of them):

placeName

placeDesc

placeAddress

Specify a list of usernames to only retrieve the posts that all of these users contributed to.
Specify a list of tags which the search engine should only return the places that are tagged with all of those tags.
Filter based on the number of people who have already been there. The user should be able to adjust the upperbound, lowerbound or only one of the two.
Specify a list of the list names that the engine should only filter the documents that have been included in all of the given list names.
Note 1: You should be aware that you should give the user the possibility to select any of the abovementioned options. How should the user use the options? We will accept any manual that you provide to the user.

Note 2: As you may have realized from 1st option, you need to build inverted indexes for those values and return all of the documents that have the similarity more than 0 concerning the given queries. Choose a logical way to aggregate the similarity coming from each of them and explain your idea in detail.

Note 3: The options other than 1st one can be considered as filtering criteria so the retrieved documents must respect all of those filters.

The output must contain the following information about the places:

placeName

placeURL

# 6. Command line question
As done in the previous assignment, we encourage using the command as a feature that Data Scientists must master.

In this question, you should use command line tools such as grep (or any other commands) to answer the following question:

For the countries Italy, Spain, France, England, and the United States, report the following (using the information scraped in point 1.3):
How many places can be found in each country?
How many people, on average, have visited the places in each country?
How many people in total want to visit the places in each country?
Note: You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable.

###### We add one column to the dataframe for each country. In this columns there are boolean values, True if the place is in that country, False otherwise.

In [261]:
df_places['england'] = df_places['placeAddress'].str.contains(pat= 'England')
df_places['Spain'] = df_places['placeAddress'].str.contains(pat='Spain')
df_places['United States'] = df_places['placeAddress'].str.contains(pat = 'United States')
df_places['France'] = df_places['placeAddress'].str.contains(pat='France')
df_places['Italy'] = df_places['placeAddress'].str.contains(pat = 'Italy')
df_places.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,...,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL,england,Spain,United States,France,Italy,England
0,City Hall Station,"['subways', 'subterranean', 'infrastructure', ...",1828,8583,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre St New York, New York, 10007 United ...",40.7134,-74.0046,...,2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",http://www.atlasobscura.com/places/city-hall-s...,False,False,True,False,False,False
1,Highgate Cemetery,"['rivals-week', 'vampires', 'hoaxes', 'occult'...",2618,8182,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, Highgate London, England, N6 Uni...",51.5675,-0.1483,...,2014-08-09,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",http://www.atlasobscura.com/places/highgate-ce...,True,False,False,False,False,True
2,Leadenhall Market,"['markets', 'film-locations', 'film']",3135,7570,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3V United Kingdom",51.5126,-0.0834,...,2016-08-01,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",http://www.atlasobscura.com/places/leadenhall-...,True,False,False,False,False,True
3,The Wave Organ,"['sea-organ', 'aletrail', 'sounds', 'music', '...",2429,7408,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green Dr San Francisco, California, ...",37.8085,-122.4401,...,2008-11-21,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",http://www.atlasobscura.com/places/wave-organ,False,False,True,False,False,False
4,Catacombes de Paris,"['ossuaries', 'memento-mori', 'catacombs-and-c...",4447,7046,"In 2004, Parisian police were assigned to do a...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-Rochereau Paris, 75014 France",48.8343,2.3322,...,2009-02-13,"['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",http://www.atlasobscura.com/places/catacombes-...,False,False,False,True,False,False


#### 6.1

###### We calculate the number of places in each country.

In [263]:
print(sum(df_places['england'] == True))
print(sum(df_places['Spain'] == True))
print(sum(df_places['United States'] == True))
print(sum(df_places['France'] == True))
print(sum(df_places['Italy'] == True))

364
91
4614
206
210


#### 6.2

##### We calculate the average of the number of visitors in each country.

In [265]:
totalEngland = sum(df_places.numPeopleVisited[df_places['england'] == True])
averageEngland = round(totalEngland/sum(df_places['england'] == True),2)
print(f" The average of visitors in England is: {averageEngland}")

totalSpain = sum(df_places.numPeopleVisited[df_places['Spain'] == True])
averageSpain = round(totalSpain/sum(df_places['Spain'] == True),2)
print(f" The average of visitors in Spain is: {averageSpain}")

totalUnitedStates = sum(df_places.numPeopleVisited[df_places['United States'] == True])
averageUnitedStates = round(totalUnitedStates/sum(df_places['United States'] == True),2)
print(f" The average of visitors in the United States is: {averageUnitedStates}")

totalFrance = sum(df_places.numPeopleVisited[df_places['France'] == True])
averageFrance = round(totalFrance/sum(df_places['France'] == True),2)
print(f" The average of visitors in France is: {averageFrance}")

totalItaly = sum(df_places.numPeopleVisited[df_places['Italy'] == True])
averageItaly = round(totalItaly/sum(df_places['Italy'] == True),2)
print(f" The average of visitors in Italy is: {averageItaly}")

 The average of visitors in England is: 475.97
 The average of visitors in Spain is: 448.45
 The average of visitors in the United States is: 436.68
 The average of visitors in France is: 425.36
 The average of visitors in Italy is: 383.3


#### 6.3

##### We calculate the total number of people who want visit the places in all the selected countries.


In [266]:
peoplewantEngland = sum(df_places.numPeopleWant[df_places.england == True])
print(f" The total number of people who want visit England is: {peoplewantEngland}")

peoplewantSpain = sum(df_places.numPeopleWant[df_places.Spain == True])
print(f" The total number of people who want visit Spain is: {peoplewantSpain}")

peoplewantUnitedStates = sum(df_places.numPeopleWant[df_places['United States'] == True])
print(f" The total number of people who want visit United States is: {peoplewantUnitedStates}")

peoplewantFrance = sum(df_places.numPeopleWant[df_places['France'] == True])
print(f" The total number of people who want visit France is: {peoplewantFrance}")

peoplewantItaly = sum(df_places.numPeopleWant[df_places['Italy'] == True])
print(f" The total number of people who want visit Italy is: {peoplewantItaly}")

 The total number of people who want visit England is: 386622
 The total number of people who want visit Spain is: 71358
 The total number of people who want visit United States is: 4340637
 The total number of people who want visit France is: 204917
 The total number of people who want visit Italy is: 182497


In [267]:
peoplewantTotal = peoplewantEngland + peoplewantSpain + peoplewantUnitedStates + peoplewantFrance + peoplewantItaly
peoplewantTotal

5186031

## 7. Theoretical question
An imaginary university is interested in accepting some of the applicants for positions to study the Master of Data Science there. Unfortunately, only a few spots are available, so the university requires students to take some exams. Students are then admitted based on how well they perform on these exams. For students to determine whether they have been successfully accepted to the university, the university wants to create a ranking list that includes every student's first name, last name, and total average on its course webpage. Students should be ranked in the list based on their average points in descending order. For example, if two students have the same average punctuation, they should be sorted in ascending order using their first and last names. University will give you the students' information in __'ApplicantsInfo.txt'__ ([click here to download](https://adm2022.s3.amazonaws.com/ApplicantsInfo.txt)), and you should provide them with the ranking list in another *.txt* file and name it as __'RankingList.txt'__ . Kindly help this university in preparing this ranking list.

**Input:** 
__'ApplicantsInfo.txt'__ will have the following format: 
- In the first line, you will be given *n* as the number of applicants and *m* as the number of exams that students have taken (all of them have taken the same exams), where: 
$$0 \lt n \le 5 * 10^4$$
$$1 \le m \le 10^3$$
- In each following *n* lines, you will find the information related to one of the students. Their first name, last name and *m* integers as the grades they received in *m* courses. 
 
**Output:**
The output file should consist of __n__ lines, with each line representing one of the students and including the student's __first name, last name, and total average point__(setting the precision to 2 decimal points). As you know, they must be sorted in the order specified in the problem description. 


1. Try solving the problem mentioned above using three different sorting algorithms (do not use any MapReduce algorithm). (__Note:__ Built-in Python functions (like .mean, .sort, etc.) are not allowed to be used. You must implement the algorithms from scratch).
2. What is the time complexity of each algorithm you have used?
3. Evaluate the time taken for each of your implementations to answer the query stored in the __ApplicantsInfo.txt__ file and visualize them.
4. What is the most optimal algorithm, in your opinion, and why?
5. Implement a sorting algorithm using MapReduce and compare it against the three algorithms previously implemented using the __ApplicantsInfo.txt__ file.

### 7.1

**Sorting Algorithms**

To sort all the rows in the ___ApplicantsInfo.txt__, we implemented 3 sorting algorithm:

1. __SelectionSort__
2. __BubbleSort__
3. __MergeSort__

In [ ]:
import time
import sys #selection sort
import os

In [ ]:
# mean rounded to 2 decimals 
def media(values,n):
    return round(sum(values)/n,2)

# unlist
def unlist(studenti):
    for i in studenti:
        print(*i) 

In [ ]:
#reading the file s.t. rows = ['name', 'surname', mean]
with open('ApplicantsInfo.txt','r',encoding='utf-8') as f:
    line = f.readline().split()
    N = int(line[0])
    M = int(line[1])
    rows = []
    for i in (range(N)):
        row = f.readline().split()
        m = media([int(x) for x in row[2:M+2]],M)
        rows.append([row[0],row[1],m]) 

In [ ]:
#how unlist fun works
unlist(rows[:3])

Richard Scott 23.83
Paula Ahmadi 24.02
Arthur Richardson 23.77

In [ ]:
# SelectionSort
def selection_sort(values,N):
    for i in range(N):
        # Searching the max element in an unsorted array
        max_i = i
        for  j in range(i+1,N):
            if( values[max_i][2] < values[j][2]):
                max_i = j
            elif (values[max_i][2] == values[j][2]):
                if((values[max_i][0]+values[max_i][1]).lower() > (values[j][0] +values[j][1]).lower()):
                    max_i = j
        values[i],values[max_i] = values[max_i],values[i]

# Swap the found max element with the first element 
def swap(i,j,values):
    temp =  values[i]
    values[i] = values[j]
    values[j] = temp



# BubbleSort
def bubble_sort(values,N):
    for i in range(N):
        for j in range(N-i-1):
            # Swap if element found > next element
            if(values[j+1][2]>values[j][2]):
                swap(j,j+1,values)
            # Alphabetical order
            elif(values[j+1][2]==values[j][2]):
                if((values[j+1][0]+values[j+1][1])<(values[j][0]+values[j][1])):
                    swap(j,j+1,values)


# MergeSort
def merge_sort(values):
    if(len(values) > 1):
        mid  = len(values)//2 #pivot as the mid of the array
        l =  values[:mid]
        r = values[mid:]
        merge_sort(l)
        merge_sort(r)
        il= 0
        ir= 0 
        i = 0
        while(il < len(l) and ir <len(r)):
            if( l[il][2] > r[ir][2]):
                values[i] = l[il]
                il +=1
            elif(l[il][2] == r[ir][2]):
                if(l[il][0]+l[il][1]).lower() < (r[ir][0] +r[ir][1]).lower():
                    values[i]  = l[il]
                    il +=1
                else:
                    values[i]  = r[ir]
                    ir +=1
            else:
                values[i] = r[ir]
                ir +=1
            i+=1
        # Checking if elements left 
        while( il < len(l)):
            values[i] = l[il]
            il += 1
            i+=1

        while( ir<len(r)):
            values[i] = r[ir]
            ir += 1
            i+=1


### 7.2 

__SelectionSort__ : $O$($N^{2}$)

Given an array of size N, in the worst case the selection sort algorithm makes for every element of the array in position i, N-i-1 comparisons (i from 0 to N-1), if array is ordered in the opposite ordering we want. Then swaps its position with the one of entry with the biggest mean in the section of the array that has not been ordered yet. 


*Number of comparisons* = $\sum_{i = 1}^{N}$ $\sum_{j=i+1}^{N} 1$ = $\sum_{i=1}^{N}(N-(i+1)+1)$ = $\sum_{i=1}^{N}(N-i)$ = $N(N-1)$ - $\frac{N(N-1)}{2}$ = $\frac{N(N-1)}{2}$ = $O$($N^{2}$)

__BubbleSort__ : $O$($N^{2}$)

Given an array of size N, in the worst case the bubble sort algorithm makes $\frac{N^{2}}{2}$ comparisons and swaps, if the array is ordered in the opposite ordering we want.

__MergeSort__ : $O$(N $\log{N}$)

Given an array of size N (with N>1). The merge sort it's a recursive algorithm:
at the first step (i=1) merges array of size 1, at the second step (i=2) merges array of size 2, at the i-th step merges array of size $2^{i-1}$.
To merge two arrays of same size N takes only one loop over the two array so it takes $O$($N$).
The last step occurs when merges array of size  $\frac{N}{2}$:

 $2^{i-1}$ = $\frac{N}{2}$   <=> i-1 = $\log{_2}{\frac{N}{2}}$ <=> i = $O$($\log{N}$).

 
So the algorithm has to merge two arrays $\log{N}$ times and it has time complexity $O$(N $\log{N}$).

### 7.3

In [ ]:
import time

In [ ]:
#1st: Merge 
s_m=time.time()
merge_sort(rows)
e_m=time.time()

print(unlist(rows[:3]))    
print('Merge does the work in ', round(e_m-s_m,2) ,'seconds')

###### Emily Crispin 24.49
###### Patricia Witten 24.49
###### Bruce Johnson 24.45
###### None
###### Merge does the work in  0.56 seconds

In [ ]:
#2nd: Selection
s_s=time.time()
selection_sort(rows,len(rows))
e_s=time.time()
print('Selection does the work in ', round(e_s-s_s,0) ,'seconds')

###### Selection does the work in  481.0 seconds

In [ ]:
#3rd: Bubble 
s_b=time.time()
bubble_sort(rows, len(rows)) 
e_b=time.time()
print('Bubble does the work in ', round(e_b-s_b,0) ,'seconds')

###### Bubble does the work in  990.0 seconds

In [ ]:
#Creating txt file with sorted students by score 
with open('RankingList.txt','w') as f:
    for i in rows: 
        f.write(i[0]+' '+ i[1] +' '+str(i[2]) +'\n')

### 7.4

The best time efficient algorithm in our experiment is the __mergeSort__.  It's a "superlinear" algorithm and its time complexity is the smallest one, being  $O$(N $\log{N}$) < $O$($N^{2}$) (bubble/selection). Follows the result of their application : gold tp __mergeSort__ with 0.52 seconds, silver to __selectionSort__ with ~ 8 minutes (481 seconds) and bronze to  __bubbleSort__ with ~ 16 minutes (990 seconds).
Even if  __bubbleSort__ and __selectionSort__ have the same time complexity, the first is worse.

### 7.5

**MapReduce Implementations**

In [ ]:
def mapReduce(values):
    c  = {}
    for elem in values:
        if (elem[2] not in c.keys()):
            c[elem[2]]=[]
        c[elem[2]].append(elem[0]+' '+elem[1])
    for k in c.keys():
        c[k] = sorted(c[k])
    return dict(sorted(c.items(),reverse=True)) 
    #{k:sorted(c[k]) for k in c.keys()}

In [ ]:
s_mr=time.time()
mr=mapReduce(rows)
e_mr = time.time()
print('MapReduce does the work in ', round(e_mr-s_mr,2) ,' seconds')

###### MapReduce does the work in  0.05  seconds

__MapReduce__ is even way faster than __BubbleSort__ with 0.05 seconds to execute the sorting.